## Implementation of Likelihood Weighting to approximate probability queries from a Bayesian Network.

In [2]:
import numpy as np
from calculations_helper import break_up_polytree, handle_dag_likelihood_weighting, join_distributions

def estimate_likelihood_weighting(iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> np.array:
    """Use likelihood weighting to estimate the probability distribution of the given query variables

    Args:
        iterations (int): number of iterations where the variables are sampled and their counts are updated accordingly
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables and their values

    Returns:
        np.array: resulting probability distribution estimate for the query variables
    """
    dag_map, query_collections, evidence_collections = break_up_polytree(network, queries, evidence)
    
    if len(dag_map) == 1:
        # only one directed acyclic graph
        return queries, handle_dag_likelihood_weighting(iterations, network, queries, evidence)
    else:
        # each directed acyclic graph will output a probability distribution - we must join them all and keep track of the variables present
        reordered_queries = []
        prob_distributions = []
        for i, dag in dag_map.items():
            these_queries = query_collections[i]
            for v in these_queries:
                reordered_queries.append(v)
            this_evidence = {v:evidence[v] for v in evidence_collections[i]}
            prob_distributions.append(handle_dag_likelihood_weighting(iterations, dag, these_queries, this_evidence))
        return reordered_queries, join_distributions(prob_distributions)

In [ ]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('small_polytree.json') as f:
        bayesian_network = json.load(f)
        start_time = time.perf_counter()
        results.append(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))
        end_time = time.perf_counter()
        times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\nProbabilities={res[1]}\nRuntime={t}\n\n")

Variables=[1, 3]
Probabilities=[6.10465120e-04 9.43689560e-01 3.16841807e-03 5.25315572e-02]
Runtime=0.304179375001695


Variables=[4, 5, 7]
Probabilities=[0.07276888 0.05038695 0.00179773 0.00103457 0.53018205 0.3208686
 0.01446551 0.00849571]
Runtime=0.3031924590031849


Variables=[2, 6, 10, 12]
Probabilities=[0.00706213 0.00418688 0.02848389 0.01688707 0.00517347 0.00306717
 0.01867208 0.01107    0.07072751 0.04193179 0.26025721 0.15429712
 0.05689278 0.03372968 0.18053092 0.10703028]
Runtime=0.37566495800274424




In [10]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('big_polytree.json') as f:
        bayesian_network = json.load(f)
        start_time = time.perf_counter()
        results.append(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))
        end_time = time.perf_counter()
        times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\nProbabilities={res[1]}\nRuntime={t}\n\n")

Variables=[1, 3]
Probabilities=[0.74542726 0.07372474 0.16433544 0.01651255]
Runtime=1.0367504169989843


Variables=[4, 5, 7]
Probabilities=[0.44812264 0.29300614 0.07230663 0.04814635 0.06847489 0.04976296
 0.01148103 0.00869936]
Runtime=1.0408804579928983


Variables=[2, 6, 10, 12]
Probabilities=[0.00607603 0.00241214 0.25474399 0.05417186 0.00997176 0.00392175
 0.40129344 0.09740975 0.00086324 0.00044587 0.05079639 0.01133025
 0.00233971 0.00065183 0.07688638 0.02668562]
Runtime=1.080366791007691




In [11]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('giant_polytree.json') as f:
        bayesian_network = json.load(f)
        start_time = time.perf_counter()
        results.append(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))
        end_time = time.perf_counter()
        times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\nProbabilities={res[1]}\nRuntime={t}\n\n")

Variables=[1, 3]
Probabilities=[0.15375167 0.78343635 0.03902016 0.02379183]
Runtime=2.121591792005347


Variables=[4, 5, 7]
Probabilities=[0.09831633 0.00250415 0.34951714 0.00707889 0.24076973 0.00767176
 0.28702032 0.00712167]
Runtime=2.0930800829955842


Variables=[2, 6, 10, 12]
Probabilities=[0.06749101 0.00551866 0.06389162 0.00379525 0.16442488 0.00907355
 0.14498453 0.0099516  0.10269336 0.00921633 0.09472353 0.00700915
 0.15927594 0.01172695 0.13459479 0.01162886]
Runtime=2.233307374990545


